# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from random import sample

import re

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV

from timeit import default_timer as timer

from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

**Note: This Step is for the extraction of California State and Foods category**

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOUSEHOLD')]

In [5]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
565,HOUSEHOLD_1_001_CA_1_validation,HOUSEHOLD_1_001,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,3,0,0,0,...,1,0,1,1,1,0,0,0,0,1
566,HOUSEHOLD_1_002_CA_1_validation,HOUSEHOLD_1_002,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,1,1,2,4,...,0,0,0,0,0,1,0,0,0,1
567,HOUSEHOLD_1_003_CA_1_validation,HOUSEHOLD_1_003,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,0,1,2,1,0,0,0,1,0
568,HOUSEHOLD_1_004_CA_1_validation,HOUSEHOLD_1_004,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,4,0,8,11,...,0,5,0,0,0,0,0,0,1,0
569,HOUSEHOLD_1_005_CA_1_validation,HOUSEHOLD_1_005,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,3,2,2,...,1,0,3,1,2,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10754,HOUSEHOLD_2_512_CA_4_validation,HOUSEHOLD_2_512,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,0,0,2,1,0,0,0,0
10755,HOUSEHOLD_2_513_CA_4_validation,HOUSEHOLD_2_513,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
10756,HOUSEHOLD_2_514_CA_4_validation,HOUSEHOLD_2_514,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,2,0,1,...,0,0,1,0,0,0,0,0,0,0
10757,HOUSEHOLD_2_515_CA_4_validation,HOUSEHOLD_2_515,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0


In [6]:
INPUT_DIR_2 = r'/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/List of Product ID according to 4 demand patterns/California/Household'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [7]:
len(sales_intermittent)

1493

In [8]:
len(sales_lumpy)

178

In [9]:
len(sales_erratic)

126

In [10]:
len(sales_smooth)

2391

**Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)**

In [11]:
sales_intermittent

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
576,HOUSEHOLD_1_012_CA_1_validation,HOUSEHOLD_1_012,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,1,1,0,0,4,0,1,2,1
577,HOUSEHOLD_1_013_CA_1_validation,HOUSEHOLD_1_013,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,0,2,0,2,2,1,0,4,1
582,HOUSEHOLD_1_018_CA_1_validation,HOUSEHOLD_1_018,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,2,2,2,1,4,1,1,0,3
599,HOUSEHOLD_1_036_CA_1_validation,HOUSEHOLD_1_036,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
600,HOUSEHOLD_1_037_CA_1_validation,HOUSEHOLD_1_037,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10750,HOUSEHOLD_2_508_CA_4_validation,HOUSEHOLD_2_508,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
10751,HOUSEHOLD_2_509_CA_4_validation,HOUSEHOLD_2_509,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,1,2,1,0,0,0,0,0,1
10756,HOUSEHOLD_2_514_CA_4_validation,HOUSEHOLD_2_514,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,2,0,1,...,0,0,1,0,0,0,0,0,0,0
10757,HOUSEHOLD_2_515_CA_4_validation,HOUSEHOLD_2_515,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0


# User-defined functions to calculate Metrics

In [12]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

# Format sales data and then merge with calendar data

In [13]:
sales_pattern = sales_intermittent.copy()

In [14]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

In [15]:
dataframe_weekly

,wm_yr_wk,HOUSEHOLD_1_001_CA_2,HOUSEHOLD_1_005_CA_3,HOUSEHOLD_1_006_CA_3,HOUSEHOLD_1_010_CA_2,HOUSEHOLD_1_010_CA_4,HOUSEHOLD_1_012_CA_1,HOUSEHOLD_1_012_CA_3,HOUSEHOLD_1_013_CA_1,HOUSEHOLD_1_013_CA_2,...,HOUSEHOLD_2_514_CA_3,HOUSEHOLD_2_514_CA_4,HOUSEHOLD_2_515_CA_1,HOUSEHOLD_2_515_CA_2,HOUSEHOLD_2_515_CA_3,HOUSEHOLD_2_515_CA_4,HOUSEHOLD_2_516_CA_1,HOUSEHOLD_2_516_CA_2,HOUSEHOLD_2_516_CA_3,HOUSEHOLD_2_516_CA_4
0,11101,0.0,1.0,0.0,0.0,0.0,4.0,0.0,3.0,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,2.0
1,11102,0.0,29.0,0.0,0.0,0.0,1.0,0.0,6.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0
2,11103,0.0,0.0,0.0,0.0,0.0,4.0,0.0,8.0,6.0,...,1.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,1.0
3,11104,0.0,10.0,0.0,0.0,0.0,3.0,0.0,5.0,5.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,11105,0.0,17.0,0.0,0.0,0.0,3.0,1.0,6.0,16.0,...,2.0,2.0,0.0,0.0,0.0,0.0,1.0,6.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,0.0,12.0,0.0,0.0,0.0,4.0,0.0,11.0,6.0,...,0.0,2.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0
269,11609,0.0,10.0,0.0,0.0,0.0,1.0,0.0,9.0,3.0,...,2.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
270,11610,0.0,12.0,0.0,0.0,0.0,4.0,0.0,13.0,0.0,...,2.0,3.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0
271,11611,1.0,18.0,0.0,0.0,0.0,6.0,0.0,15.0,0.0,...,1.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0


In [16]:
lags = list(range(1, 25))
moving_averages = list(range(1, 25)) 
stds = list(range(2, 25))
maximums = list(range(1, 25))
minimums = list(range(1, 25))
totals = list(range(1, 25))
num_days_zeros = list(range(1, 25))

correlation_threshold = 0.8

end_train_week = 11610

num_id_sample_for_tuning = 200

In [17]:
sample_list = sample(list_pattern_py, num_id_sample_for_tuning)

In [18]:
dataframe_check = dataframe_weekly[sample_list].iloc[-2:].T

dataframe_check['Check'] = np.where((dataframe_check[271] == 0) &
                                    (dataframe_check[272] == 0), 1, 0)

sample_removed = dataframe_check[dataframe_check['Check'] == 1].index.tolist()

In [19]:
sample_list_filtered = [x for x in sample_list if (x not in sample_removed)]

In [20]:
len(sample_list_filtered)

129

In [ ]:
start = timer()

df_output_hyper_params = pd.DataFrame()
    
for product in sample_list_filtered:
    print('Currently Running: %s' % product)
    print('Progressing: {0} %'.format(round(sample_list_filtered.index(product) / len(sample_list_filtered) * 100, 2)))

    dataframe_product = dataframe_weekly[['wm_yr_wk', product]]

    # Create Lag variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (t-{lag})': dataframe_product[product].shift(lag)
        for lag in lags
    })

    # Create Rolling Moving Average variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
        for moving_average in moving_averages
    })

    # Create Rolling Standard Deviation variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
        for std in stds
    })

    # Create Rolling Maximum variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
        for maximum in maximums
    })

    # Create Rolling Minimum variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
        for minimum in minimums
    })

    # Create Rolling Total variables 
    dataframe_product = dataframe_product.assign(**{
        f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
        for total in totals
    })


    dataframe_product.dropna(inplace=True)
    dataframe_product.set_index('wm_yr_wk', inplace=True)

    train_data = dataframe_product[dataframe_product.index <= end_train_week]

    corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
    high_corr_var=np.where(corr_matrix > correlation_threshold)
    high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]

    train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)

    X_train = train_data.drop(product, axis=1)
    y_train = train_data[product]  

    tscv = TimeSeriesSplit(n_splits=3)

    params = {'n_estimators': [30, 50],
              'max_depth': list(range(3,11)),
              'min_samples_leaf': range(4,11),
              'max_features': ['sqrt', 'log2', None],
              'bootstrap': [True], #We force bootstrap
              'max_samples': [0.6, 0.7, 0.8, 0.9]}

    forest = RandomForestRegressor(n_jobs=1)

    regressor = RandomizedSearchCV(estimator=forest,
                                  param_distributions=params,
                                  cv = tscv, 
                                  scoring='neg_mean_absolute_percentage_error',
                                  n_iter=150,
                                  verbose=1)
    regressor.fit(X_train, y_train)
    best_params_temp = regressor.best_params_

    df_output_temp = pd.DataFrame.from_dict(best_params_temp, orient='index').T
    df_output_temp['Product'] = product

    df_output_hyper_params = df_output_hyper_params.append(df_output_temp, ignore_index=True)

    
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: HOUSEHOLD_2_105_CA_1
Progressing: 0.0 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: HOUSEHOLD_1_260_CA_3
Progressing: 0.78 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: HOUSEHOLD_2_187_CA_1
Progressing: 1.55 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: HOUSEHOLD_1_092_CA_3
Progressing: 2.33 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: HOUSEHOLD_2_211_CA_2
Progressing: 3.1 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: HOUSEHOLD_2_259_CA_4
Progressing: 3.88 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: HOUSEHOLD_2_445_CA_2
Progressing: 4.65 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running: HOUSEHOLD_1_186_CA_1
Progressing: 5.43 %
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Currently Running:

In [ ]:
df_output_hyper_params.to_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/Random Forest (Completed Pipeline)/Foods/Intermittent_Params_Tuning_Random Forest.csv')

# Decide optimum set of parameters for Random Forest algorithm

In [ ]:
# See if any values larger than 1 in column "Product"
# If yes, the corresponding set of parameters occur more than once and those sets should be considered first
df_output_hyper_params.groupby(['n_estimators', 'max_depth', 'min_samples_leaf', 'max_features', 'max_samples'])[['Product']].count().sort_values(by = 'Product', ascending=False)

Product
n_estimators max_depth min_samples_leaf max_features max_samples         
30           6         4                log2         0.6                2
             3         5                log2         0.7                1
             8         7                sqrt         0.8                1
50           10        5                sqrt         0.6                1
             9         4                log2         0.8                1
             5         4                sqrt         0.9                1
             4         4                log2         0.7                1
             3         7                log2         0.8                1
30           10        10               log2         0.9                1
                       9                sqrt         0.7                1
                       6                log2         0.9                1
             9         7                sqrt         0.6                1
                       6                sqrt         0.7                1
                       5                log2         0.9                1
             8         8                sqrt         0.6                1
                                        log2         0.7                1
                       7                sqrt         0.6                1
             3         5                sqrt         0.8                1
             8         4                log2         0.6                1
             7         10               log2         0.7                1
             5         7                log2         0.7                1
                       6                log2         0.9                1
                       5                sqrt         0.8                1
                                                     0.6                1
                                        log2         0.6                1
             4         10               log2         0.8                1
                       6                sqrt         0.9                1
                       4                sqrt         0.9                1
             3         10               log2         0.9                1
                       6                sqrt         0.7                1
50           10        9                log2         0.7                1

In [ ]:
df_output_hyper_params.groupby(['n_estimators'])[['Product']].count()

,Product
n_estimators,
30,105
50,24


In [ ]:
df_output_hyper_params.groupby(['max_depth'])[['Product']].count()

,Product
max_depth,
3,15
4,12
5,16
6,17
7,14
8,22
9,15
10,18


In [ ]:
df_output_hyper_params.groupby(['min_samples_leaf'])[['Product']].count()

,Product
min_samples_leaf,
4,35
5,24
6,17
7,17
8,13
9,12
10,11


In [ ]:
df_output_hyper_params.groupby(['max_features'])[['Product']].count()

,Product
max_features,
log2,18
sqrt,14


In [ ]:
df_output_hyper_params.groupby(['max_samples'])[['Product']].count()

,Product
max_samples,
0.6,28
0.7,37
0.8,25
0.9,39


# Implement Random Forest on Test Set with optimum set of parameters

### Specify optimum set of parameters

In [ ]:
n_estimators_chosen = 30
max_depth_chosen = 4
min_samples_leaf_chosen = 5
max_features_chosen = 'log2'
max_samples_chosen = 0.9

In [ ]:
len(list_pattern_py)

1493

In [ ]:
list_pattern_temp = list_pattern_py.copy()

In [ ]:
start = timer()

df_output_rf_pattern = pd.DataFrame()
for product in list_pattern_temp:
  print('Currently Running: %s' % product)
  print('Progressing: {0} %'.format(round(list_pattern_temp.index(product) / len(list_pattern_temp) * 100, 2)))
  dataframe_product = dataframe_weekly[['wm_yr_wk', product]]
  # Create Lag variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (t-{lag})': dataframe_product[product].shift(lag)
      for lag in lags
  })

  # Create Rolling Moving Average variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
      for moving_average in moving_averages
  })

  # Create Rolling Standard Deviation variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
      for std in stds
  })

  # Create Rolling Maximum variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
      for maximum in maximums
  })

  # Create Rolling Minimum variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
      for minimum in minimums
  })

  # Create Rolling Total variables 
  dataframe_product = dataframe_product.assign(**{
      f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
      for total in totals
  })

  dataframe_product.dropna(inplace=True)
  dataframe_product.set_index('wm_yr_wk', inplace=True)

  train_data = dataframe_product[dataframe_product.index <= end_train_week]
  test_data = dataframe_product[dataframe_product.index > end_train_week]

  corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
  high_corr_var=np.where(corr_matrix > correlation_threshold)
  high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]

  train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
  test_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)

  X_train = train_data.drop(product, axis=1)
  y_train = train_data[product]

  X_test = test_data.drop(product, axis=1)
  y_test = test_data[product]

  forest = RandomForestRegressor(n_jobs=1,
                                n_estimators = n_estimators_chosen,
                                min_samples_leaf = min_samples_leaf_chosen,
                                max_samples = max_samples_chosen,
                                max_features = max_features_chosen,
                                max_depth = max_depth_chosen,
                                bootstrap = [True],
                                random_state = 0)
  

  forest.fit(X_train,y_train)
  y_pred = forest.predict(X_test)
  df_output_temp = pd.DataFrame({'Actual Data': y_test, 
                                  'Forecast': y_pred, 
                                  'Product': [product for num_rows in range(len(y_pred))]
                                  })
  
  df_output_rf_pattern = df_output_rf_pattern.append(df_output_temp, ignore_index=False)  

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

In [ ]:
df_output_rf_pattern

,Actual Data,Forecast,Product
wm_yr_wk,,,
11611,6.0,3.226715,HOUSEHOLD_1_012_CA_1
11612,8.0,2.752293,HOUSEHOLD_1_012_CA_1
11611,15.0,8.706118,HOUSEHOLD_1_013_CA_1
11612,6.0,10.574552,HOUSEHOLD_1_013_CA_1
11611,0.0,0.115055,HOUSEHOLD_1_018_CA_1
...,...,...,...
11612,0.0,0.915225,HOUSEHOLD_2_514_CA_4
11611,0.0,0.052228,HOUSEHOLD_2_515_CA_4
11612,0.0,0.052228,HOUSEHOLD_2_515_CA_4


In [ ]:
df_output_rf_pattern.to_csv('Intermittent_Random_Forest_Forecast_Values.csv')

In [ ]:
# User-defined funcion to calculate metrics

def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [ ]:
# Start calculating metrics 
start = timer()

df_forecast_metrics = df_output_rf_pattern.groupby('Product').apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.04519358670000505 minutes


In [ ]:
df_forecast_metrics

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_001_CA_2,0.514472,102.8944,162.6060,77.0033,0.514472,0.574450
1,HOUSEHOLD_1_005_CA_3,2.211344,44.2269,56.2998,43.8339,8.845378,9.000289
2,HOUSEHOLD_1_006_CA_3,inf,inf,200.0000,NaN,0.452650,0.453346
3,HOUSEHOLD_1_010_CA_2,inf,inf,200.0000,NaN,0.104595,0.104595
4,HOUSEHOLD_1_010_CA_4,inf,inf,200.0000,NaN,0.055995,0.055995
...,...,...,...,...,...,...,...
1488,HOUSEHOLD_2_515_CA_4,inf,inf,200.0000,NaN,0.052228,0.052228
1489,HOUSEHOLD_2_516_CA_1,0.626484,125.2968,134.4974,51.2982,1.252968,1.273365
1490,HOUSEHOLD_2_516_CA_2,inf,inf,200.0000,NaN,1.675904,1.690511
1491,HOUSEHOLD_2_516_CA_3,0.409038,81.8075,107.9962,14.8083,0.409038,0.485190


In [ ]:
df_forecast_metrics.to_csv('Random_Forest_Intermittent_Forecast_Metrics.csv')